In [ ]:
import os
import sys
import json
import shutil
import pandas as pd
import nibabel as nib
from quantification_functions import crop_TOF_and_seg, get_LSA_with_MCA, postprocess_LSA_seg, get_all_LSA_metrics

## II. LSA Quantification

### 1. Define LSA ROI (manual in Slicer) 
Save the index location of the selected LSA ROI to `LSA_ROI_location.json`.

Run step 2-6 for each side of the brain.

### 2. Crop segmentation and label islands

In [ ]:
############# TODO: change input paths #############
ID = "001"      # ID number
side = "Right"  # "Left" or "Right"
root_dir = "/path/to/LUMEN/Quantification"
data_dir = os.path.join("/path/to/data/folder", ID) 
TOF_path = os.path.join(data_dir, f"{ID}_upsampled_TOF.nii.gz") # ensure that this is the isotropically sampled TOF
seg_path = os.path.join(data_dir, f"{ID}_seg.nii.gz")           # segmentation mask from DS6
#################################
LSA_ROI_dict_path = os.path.join(root_dir, "LSA_ROI_location.json")
mip_volume_property_path = os.path.join(root_dir, "MIP_VolumeProperty.vp")
LSA_ROI_dict = json.load(open(LSA_ROI_dict_path, "r"))
save_dir = os.path.join(data_dir, side)

In [ ]:
cropped_TOF_path, cropped_seg_path = crop_TOF_and_seg(ID, 
                                                    side, 
                                                    save_dir, 
                                                    TOF_path, 
                                                    seg_path, 
                                                    LSA_ROI_dict, 
                                                    auto_remove_distant_islands=False
                                                    ) # If this step takes too long or the wrong islands are removed, set `auto_remove_distant_islands=False`.

### 3. Manual correction and keep LSAs
#### 3.1 Manual correction
- Launch Slicer. Load the cropped TOF image, `labelled_seg.nii.gz` and `helper_endpoints.json` into Slicer. 

- Go to `Segment Editor` module and correct the segmentation of the LSAs. Note that **you only need to check the segmentation of the LSAs and the part of the main artery that they are connected to**. You do not need to manually remove all non-LSA islands, as in the next step only the largest island will be retained.

- Tip: Clicking on the endpoints in 3D view can take you to those locations directly in the slice views.

- Save corrected segmentation to default filename in the same folder, `labelled_seg.nii.gz.nii.seg.nrrd`.

#### 3.2 Keep LSA Segmentation  
Run the code below to create a segmentation file containing only the LSAs and MCA, `LSA_MCA_seg.nii.gz`.

In [ ]:
corrected_nrrd_path = os.path.join(save_dir, "labelled_seg.nii.gz.nii.seg.nrrd")
LSA_MCA_seg_path = get_LSA_with_MCA(corrected_nrrd_path, cropped_seg_path, save_dir)

Load `LSA_MCA_seg.nii.gz` into Slicer, and cut off the MCA. Save the segmentation to default path, `LSA_MCA_seg.nii.gz.nii.seg.nrrd`.

### 4. Postprocessing and endpoint detection  
Run the code below to postprocess the segmentation for LSAs. It will create a new folder named `Postprocessed_LSA_seg`, in which there will be the postprocessed segmentation mask with the LSA branches originating from different stems labelled separately, as well as json files storing the detected endpoints for each cluster of branches. You may load these files into Slicer again to check if the detected endpoints and their corresponding segmentation are correct. If modifying the endpoints, save them to the same json files. If modifying the segmentation, save it to the default filename (`postprocessed_LSA_seg.nii.gz.nii.seg.nrrd`).

In [ ]:
LSA_nrrd_path = os.path.join(save_dir, "LSA_MCA_seg.nii.gz.nii.seg.nrrd")
postprocessed_save_dir = os.path.join(save_dir, "Postprocessed_LSA_seg")
postprocessed_LSA_seg_filename = "postprocessed_LSA_seg.nii.gz"

# Remove previous results if they exist
if os.path.exists(postprocessed_save_dir):
    shutil.rmtree(postprocessed_save_dir)

postprocessed_LSA_seg_path = postprocess_LSA_seg(LSA_nrrd_path, ref_img_path=cropped_seg_path, 
                                                save_dir=postprocessed_save_dir, 
                                                out_seg_name=postprocessed_LSA_seg_filename)

### 5. Centreline extraction and construct LSA trees
Run the code below to automatically launch Slicer, extract the centrelines of the LSA segmentation, and save them to a new `Results` folder. If the results look right, simply close the window or stop this cell.
P.S.  This step may run for a few minutes. If you are re-running this step and already has a `Results` folder, this will over-write the result files!

In [ ]:
results_dir = os.path.join(save_dir, "Results")
if os.path.exists(results_dir):
    shutil.rmtree(results_dir)

# Run extract centreline from Slicer
# TODO: replace /path/to/slicer/on/your/computer with your own path to Slicer
# on Mac it might be "/Applications/Slicer.app/Contents/MacOS/Slicer"; on Windows it might be something like "C:\"Slicer 5.6.1"\Slicer.exe"
!/path/to/slicer/on/your/computer --no-splash --python-script extract_centerline_in_slicer.py -load_dir {postprocessed_save_dir} -seg_filename {postprocessed_LSA_seg_filename} -TOF_path {cropped_TOF_path} -results_dir {results_dir} -mip_volume_property {mip_volume_property_path}

### 6. Compute LSA metrics  
Run the code below to compute morphological metrics of the LSAs.

In [ ]:
metrics = ['length', 'tortuosity', 'mean_curvature', 'max_curvature']
summary_results, full_results = get_all_LSA_metrics(results_dir, metrics) # full_results contains all the metrics for each vessel segment and branch

df = pd.DataFrame([summary_results]).T
df.columns = ['Value']
display(df)